In [ ]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License

# Use windowing with RunInference predictions 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/beam-ml/run_inference_windowing.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/apache/beam/blob/master/examples/notebooks/beam-ml/run_inference_windowing.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png" />View source on GitHub</a>
  </td>
</table>


This notebook shows how to use the RunInference transform with [windowing](https://beam.apache.org/documentation/programming-guide/#windowing) in a streaming pipeline. Windowing is useful when your data arrives within a particular timeframe and can be divided by timestamp, or when you want to see trends before all the data is processed. In this example, the pipeline predicts the quality of milk samples and classifies them as `good`, `bad`, or `medium`. The pipeline then aggregates the predictions for each window. To make predictions, the pipeline uses the XGBoost model handler. For more information about the RunInference API, see the [Machine Learning section of the Apache Beam documentation](https://beam.apache.org/documentation/ml/overview/).

With RunInference, a model handler manages batching, vectorization, and prediction optimization for your XGBoost pipeline or model.

This notebook demonstrates the following common RunInference patterns:

- Generate predictions for all samples in a window.
- Aggregate the results per window after running inference.
- Print the aggregations.

## Before you begin
Complete the following setup steps:
- Install dependencies for Apache Beam.
- Install XGBoost.
- Download the [Milk Quality Prediction dataset from Kaggle](https://www.kaggle.com/datasets/cpluzshrijayan/milkquality). Name the dataset `milk_quality.csv`, and put it in the current directory. Use the CSV file format for the dataset.

In [ ]:
!pip install git+https://github.com/apache/beam.git
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/apache/beam.git to /tmp/pip-req-build-4rrh40ti
  Running command git clone --filter=blob:none --quiet https://github.com/apache/beam.git /tmp/pip-req-build-4rrh40ti
  Resolved https://github.com/apache/beam.git to commit 47f1123f03f52744b951a8b6fa067b32e66112bf
  Running command git submodule update --init --recursive -q
ERROR: git+https://github.com/apache/beam.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


## About the dataset

This dataset comes in the form of a csv file that consists of 7 columns: pH, temperature, taste, odor, fat, turbidity, and color. The dataset also contains a column that labels the quality of that sample as `good`, `bad` or `medium`.

In [ ]:
import argparse
import logging
import time
from typing import NamedTuple

import pandas
from sklearn.model_selection import train_test_split

import apache_beam as beam
import xgboost
from apache_beam import window
from apache_beam.ml.inference import RunInference
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference.xgboost_inference import XGBoostModelHandlerPandas
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.runners.runner import PipelineResult
from apache_beam.testing.test_stream import TestStream

ModuleNotFoundError: ignored

## Load the Kaggle dataset and train XGBoost model
This section demonstrates the following steps:
1. Load the Milk Quality dataset from Kaggle.
2. Split the data in a training and test set.
2. Train the XGBoost classifier to predict the quality of milk.
3. Save the model in a JSON file using `mode.save_model`. (https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html)


In [ ]:
DATASET = "dataset.csv"
TRAINING_SET = "training_set.csv"
TEST_SET = "test_set.csv"
LABELS = "labels.csv"
MODEL_STATE = "model.json"

### Preprocessing helper functions

In [ ]:
def preprocess_data(
    dataset_path: str,
    training_set_path: str,
    labels_path: str,
    test_set_path: str):
  """
    Helper function to split the dataset into a training set
    and its labels and a test set. The training set and
    its labels are used to train a lightweight model.
    The test set is used to create a test streaming pipeline.
    Args:
        dataset_path: path to csv file containing the Kaggle
         milk quality dataset
        training_set_path: path to output the training samples
        labels_path:  path to output the labels for the training set
        test_set_path: path to output the test samples
    """
  df = pandas.read_csv(dataset_path)
  df['Grade'].replace(['low', 'medium', 'high'], [0, 1, 2], inplace=True)
  x = df.drop(columns=['Grade'])
  y = df['Grade']
  x_train, x_test, y_train, _ = \
      train_test_split(x, y, test_size=0.60, random_state=99)
  x_train.to_csv(training_set_path, index=False)
  y_train.to_csv(labels_path, index=False)
  x_test.to_csv(test_set_path, index=False)


def train_model(
    samples_path: str, labels_path: str, model_state_output_path: str):
  """Function to train the XGBoost model.
    Args:
      samples_path: path to csv file containing the training data
      labels_path: path to csv file containing the labels for the training data
      model_state_output_path: Path to store the trained model
  """
  samples = pandas.read_csv(samples_path)
  labels = pandas.read_csv(labels_path)
  xgb = xgboost.XGBClassifier(max_depth=3)
  xgb.fit(samples, labels)
  xgb.save_model(model_state_output_path)
  return xgb

### Preprocess the data and train the model

We split the dataset in a training set, test set and labels for training set. We use the test set as input data for our test stream, you can use the test set to validate the trained model's performance. After preprocessing, we have 3 different files containing data. Once we have our training set, we can also train the XGBoost model and store it in a json file, such that it can be loaded by the ModelHandler.

In [ ]:
preprocess_data(
    dataset_path=DATASET,
    training_set_path=TRAINING_SET,
    labels_path=LABELS,
    test_set_path=TEST_SET)

train_model(
    samples_path=TRAINING_SET,
    labels_path=LABELS,
    model_state_output_path=MODEL_STATE)

In [ ]:
# Named tuple to store the number of good, bad and medium quality samples in a window
class MilkQualityAggregation(NamedTuple):
  bad_quality_measurements: int
  medium_quality_measurements: int
  high_quality_measurements: int

NameError: ignored

### Helper CombineFn to aggregate the results of a window, the function keeps track of the number of good, bad and medium quality samples in the stream 

In [ ]:
class AggregateMilkQualityResults(beam.CombineFn):
  """Simple aggregation to keep track of the number
   of samples with good, bad and medium quality milk."""
  def create_accumulator(self):
    return MilkQualityAggregation(0, 0, 0)

  def add_input(
      self, accumulator: MilkQualityAggregation, element: PredictionResult):
    quality = element.inference[0]
    if quality == 0:
      return MilkQualityAggregation(
          accumulator.bad_quality_measurements + 1,
          accumulator.medium_quality_measurements,
          accumulator.high_quality_measurements)
    elif quality == 1:
      return MilkQualityAggregation(
          accumulator.bad_quality_measurements,
          accumulator.medium_quality_measurements + 1,
          accumulator.high_quality_measurements)
    else:
      return MilkQualityAggregation(
          accumulator.bad_quality_measurements,
          accumulator.medium_quality_measurements,
          accumulator.high_quality_measurements + 1)

  def merge_accumulators(self, accumulators: MilkQualityAggregation):
    return MilkQualityAggregation(
        sum(
            aggregation.bad_quality_measurements
            for aggregation in accumulators),
        sum(
            aggregation.medium_quality_measurements
            for aggregation in accumulators),
        sum(
            aggregation.high_quality_measurements
            for aggregation in accumulators),
    )

  def extract_output(self, accumulator: MilkQualityAggregation):
    return accumulator

### Create a streaming pipeline using the test data

We construct a TestStream that contains all samples from the test set.

In [ ]:
milk_quality_data = pandas.read_csv(TEST_SET)

start = time.mktime(time.strptime('2023/06/29 10:00:00', '%Y/%m/%d %H:%M:%S'))

# Create a test stream
test_stream = TestStream()

# Watermark is set to 10:00:00
test_stream.advance_watermark_to(start)

# Split the dataframe in individual samples
samples = [
    milk_quality_data.iloc[i:i + 1] for i in range(len(milk_quality_data))
]

for watermark_offset, sample in enumerate(samples, 1):
  test_stream.advance_watermark_to(start + watermark_offset)
  test_stream.add_elements([sample])

test_stream.advance_watermark_to_infinity()

### Create the XGBoost model handler and run the pipeline

In [ ]:
model_handler = XGBoostModelHandlerPandas(
    model_class=xgboost.XGBClassifier, model_state=MODEL_STATE)

pipeline_options = PipelineOptions().from_dictionary({})

with beam.Pipeline() as p:
  _ = (
      p | test_stream
      | 'window' >> beam.WindowInto(window.SlidingWindows(30, 5))
      | "RunInference" >> RunInference(model_handler)
      | 'Count number of elements in window' >> beam.CombineGlobally(
          AggregateMilkQualityResults()).without_defaults()
      | 'Print' >> beam.Map(print))